In [ ]:
import os 
import json
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from utils import *

# Loading Data 

In [ ]:
class House: 
  def __init__(self, address, postal_code, type, real_price, 
                size, basement_size, rooms, year_built, 
                year_rebuilt, energy_label, image_floorplan): 
    
    #Textual Data 
    self.address = address  
    self.postal_code = postal_code
    self.type = type
    self.price = real_price
    self.size = size
    self.basement_size = basement_size
    self.rooms = rooms
    self.year_built = year_built
    self.year_rebuilt = year_rebuilt
    self.energy_label = energy_label

    #Image Data 
    self.image_floorplan = image_floorplan
    
    #Predictions 
    self.predicted_price = None


def load_jpg_and_json(folder_path:str) -> (dict, np.ndarray):
  files = os.listdir(folder_path)
  jpg_file_path = None
  json_file_path = None

  # Find the jpg and json file in the folder
  for filename in files:
    if filename.endswith(".jpg"):
      jpg_file_path = os.path.join(folder_path, filename)
    elif filename.endswith(".json"):
      json_file_path = os.path.join(folder_path, filename)

  # Load the jpg
  image_data = cv2.imread(jpg_file_path)
  # Load the json
  with open(json_file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)

  if image_data is None:
    raise Exception(f"Error loading image {jpg_file_path}")
  if json_data is None:
    raise Exception(f"Error loading json {json_file_path}")

  return json_data, image_data

def create_house_instance(json_data, jpg): 
  address = json_data["address"]
  postal_code = json_data["postal_code"]
  type = json_data["type"]
  price = json_data["price"]
  size = json_data["size"]
  basement_size = json_data["basement_size"]
  rooms = json_data["rooms"]
  year_built = json_data["year_built"]
  year_rebuilt = json_data["year_rebuilt"] if json_data["year_rebuilt"] else None
  energy_label = json_data["energy_label"]
  image_floorplan = jpg

  house = House(address, postal_code, type, price, 
                size, basement_size, rooms, year_built, 
                year_rebuilt, energy_label, image_floorplan)
  return house

def load_houses(folder_path: str, max_houses: int = None):
    houses = []
    count = 0  # Counter to track the number of loaded houses
    for folder in os.listdir(folder_path):
        if max_houses is not None and count >= max_houses:
            break  # Stop loading houses if the maximum number is reached
        try:
            json_data, jpg = load_jpg_and_json(os.path.join(folder_path, folder))
            house = create_house_instance(json_data, jpg)
            houses.append(house)
            count += 1
        except Exception as e:
            print(f"Error loading house {folder}: {e}")
    return houses

#If we want to work with a DF 
def data_to_DF(houses: list[House])-> pd.DataFrame:
  data = []
  for house in houses:
    data.append([house.address, house.postal_code, house.type, house.price, 
                house.size, house.basement_size, house.rooms, house.year_built, 
                house.year_rebuilt, house.energy_label, house.image_floorplan])
  df = pd.DataFrame(data, columns = ["address", "postal_code", "type", "price", 
                "size", "basement_size", "rooms", "year_built", 
                "year_rebuilt", "energy_label", "image_floorplan"])
  return df

In [ ]:
path = "../nybolig-scrape/output"
houses = load_houses(path, max_houses=1000)
houses_df = data_to_DF(houses)
display(houses_df.head())

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train, validation and test sets with a 60-20-20 ratio
train_df, test_df = train_test_split(houses_df, test_size=0.2, random_state=0)
train_df, valid_df = train_test_split(train_df, test_size=0.20, random_state=0)

In [ ]:
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(zip(train_df["image_floorplan"][0:9], train_df["price"][0:9])):
    plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(f"{label}")
    plt.axis("off")

# Pre-trained VGG16

In [ ]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

In [ ]:
import tensorflow as tf

# Check available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Set the GPU to be used
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU available")

In [ ]:
# Reshape the images to the desired size
train_images = np.array([cv2.resize(image, (500, 500)) for image in train_df["image_floorplan"]])
valid_images = np.array([cv2.resize(image, (500, 500)) for image in valid_df["image_floorplan"]])
test_images = np.array([cv2.resize(image, (500, 500)) for image in test_df["image_floorplan"]])

In [ ]:
plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(zip(train_images[0:9], train_df["price"][0:9])):
    plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(f"{label}")
    plt.axis("off")

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Load pre-trained VGG16 model (without including top layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(500, 500, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add new top layers for regression
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation="relu"),
    Dense(256, activation="relu"),
    Dense(1, activation="linear")
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_absolute_error')
model.summary()

In [ ]:
# Train the model
history = model.fit(train_images, train_df["price"], validation_data=(valid_images, valid_df["price"]), epochs=2)

In [ ]:
model.evaluate(test_images, test_df["price"])

# Make predictions
predictions = model.predict(test_images)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

real_prices = test_df['price'].values
predicted_prices = predictions.flatten()

# Print the R2 score, MAE and MSE
print(f"R2 score: {r2_score(real_prices, predicted_prices):.2f}")
print(f"Mean Absolute Error: {mae(real_prices, predicted_prices):.2f}")
print(f"Mean Squared Error: {mse(real_prices, predicted_prices):.2f}")

# Plot the predictions
plt.figure(figsize=(10, 10))
for i, (image, label, prediction) in enumerate(zip(test_images[0:9], test_df["price"][0:9], predictions[0:9])):
    plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(f"Real: {label}\nPredicted: {prediction[0]:.0f}")
    plt.axis("off")
plt.show()

# Plot the predictions vs real prices
plot_regression_results('VGG16', real_prices, predicted_prices)